In [ ]:
# https://github.com/modelscope/DiffSynth-Studio/blob/main/examples/Diffutoon/Diffutoon.ipynb modified

%cd /content
!git clone https://github.com/Artiprocher/DiffSynth-Studio
%cd /content/DiffSynth-Studio

!pip install -q einops transformers controlnet-aux==0.0.7 sentencepiece imageio imageio-ffmpeg

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/229575 -d /content/DiffSynth-Studio/models/stable_diffusion -o aingdiffusion_v12.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt -d /content/DiffSynth-Studio/models/AnimateDiff -o mm_sd_v15_v2.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth -d /content/DiffSynth-Studio/models/ControlNet -o control_v11p_sd15_lineart.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth -d /content/DiffSynth-Studio/models/ControlNet -o control_v11f1e_sd15_tile.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth -d /content/DiffSynth-Studio/models/ControlNet -o control_v11f1p_sd15_depth.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth -d /content/DiffSynth-Studio/models/ControlNet -o control_v11p_sd15_softedge.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/dpt_hybrid-midas-501f0c75.pt -d /content/DiffSynth-Studio/models/Annotators -o dpt_hybrid-midas-501f0c75.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/ControlNetHED.pth -d /content/DiffSynth-Studio/models/Annotators -o ControlNetHED.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/sk_model.pth -d /content/DiffSynth-Studio/models/Annotators -o sk_model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/sk_model2.pth -d /content/DiffSynth-Studio/models/Annotators -o sk_model2.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/25820?type=Model&format=PickleTensor&size=full&fp=fp16" -d /content/DiffSynth-Studio/models/textual_inversion -o verybadimagenegative_v1.3.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/Diffutoon/resolve/main/input_video.mp4 -d /content -o input_video.mp4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil
import os

# 定义源文件路径和目标文件路径
source_file = '/content/drive/MyDrive/test_auto_mv/input_video333.mp4'
destination_dir = '/content'
destination_file = os.path.join(destination_dir, 'input_video.mp4')

# 确保目标目录存在，如果不存在则创建
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# 移动并重命名文件
shutil.move(source_file, destination_file)

print(f'File moved to {destination_file}')

In [ ]:
config_stage_1_template = {
    "models": {
        "model_list": [
            "models/stable_diffusion/aingdiffusion_v12.safetensors",
            "models/ControlNet/control_v11p_sd15_softedge.pth",
            "models/ControlNet/control_v11f1p_sd15_depth.pth"
        ],
        "textual_inversion_folder": "models/textual_inversion",
        "device": "cuda",
        "lora_alphas": [],
        "controlnet_units": [
            {
                "processor_id": "softedge",
                "model_path": "models/ControlNet/control_v11p_sd15_softedge.pth",
                "scale": 0.5
            },
            {
                "processor_id": "depth",
                "model_path": "models/ControlNet/control_v11f1p_sd15_depth.pth",
                "scale": 0.5
            }
        ]
    },
    "data": {
        "input_frames": {
            "video_file": "/content/input_video.mp4",
            "image_folder": None,
            "height": 720,
            "width": 480,
            "start_frame_id": 0,
            "end_frame_id": 30
        },
        "controlnet_frames": [
            {
                "video_file": "/content/input_video.mp4",
                "image_folder": None,
                "height": 720,
                "width": 480,
                "start_frame_id": 0,
                "end_frame_id": 30
            },
            {
                "video_file": "/content/input_video.mp4",
                "image_folder": None,
                "height": 720,
                "width": 480,
                "start_frame_id": 0,
                "end_frame_id": 30
            }
        ],
        "output_folder": "data/examples/diffutoon_edit/color_video",
        "fps": 25
    },
    "smoother_configs": [
        {
            "processor_type": "FastBlend",
            "config": {}
        }
    ],
    "pipeline": {
        "seed": 0,
        "pipeline_inputs": {
            "prompt": "best quality, perfect anime illustration, orange clothes, night, a girl is dancing, smile, solo, black silk stockings",
            "negative_prompt": "verybadimagenegative_v1.3",
            "cfg_scale": 7.0,
            "clip_skip": 1,
            "denoising_strength": 0.9,
            "num_inference_steps": 20,
            "animatediff_batch_size": 8,
            "animatediff_stride": 4,
            "unet_batch_size": 8,
            "controlnet_batch_size": 8,
            "cross_frame_attention": True,
            "smoother_progress_ids": [-1],
            # The following parameters will be overwritten. You don't need to modify them.
            "input_frames": [],
            "num_frames": 30,
            "width": 480,
            "height": 720,
            "controlnet_frames": []
        }
    }
}

from diffsynth import SDVideoPipelineRunner

config_stage_1 = config_stage_1_template.copy()
config_stage_1["data"]["input_frames"] = {
    "video_file": "/content/input_video.mp4",
    "image_folder": None,
    "height": 720,
    "width": 480,
    "start_frame_id": 0,
    "end_frame_id": 30
}
config_stage_1["data"]["controlnet_frames"] = [config_stage_1["data"]["input_frames"], config_stage_1["data"]["input_frames"]]
config_stage_1["data"]["output_folder"] = "/content/color_video"
config_stage_1["data"]["fps"] = 25
config_stage_1["pipeline"]["pipeline_inputs"]["prompt"] = "best quality, perfect anime illustration, orange clothes, night, a girl is dancing, smile, solo, black silk stockings"

runner = SDVideoPipelineRunner()
runner.run(config_stage_1)

In [ ]:
import shutil
import os

# 定义源文件路径和目标文件路径
source_file = '/content/color_video/video.mp4'
destination_dir = '/content/drive/MyDrive/test_auto_mv'
destination_file = os.path.join(destination_dir, 'video1.mp4')

# 确保目标目录存在，如果不存在则创建
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# 移动并重命名文件
shutil.move(source_file, destination_file)

print(f'File moved to {destination_file}')

In [ ]:
config_stage_1_template = {
    "models": {
        "model_list": [
            "models/stable_diffusion/aingdiffusion_v12.safetensors",
            "models/ControlNet/control_v11p_sd15_softedge.pth",
            "models/ControlNet/control_v11f1p_sd15_depth.pth"
        ],
        "textual_inversion_folder": "models/textual_inversion",
        "device": "cuda",
        "lora_alphas": [],
        "controlnet_units": [
            {
                "processor_id": "softedge",
                "model_path": "models/ControlNet/control_v11p_sd15_softedge.pth",
                "scale": 0.5
            },
            {
                "processor_id": "depth",
                "model_path": "models/ControlNet/control_v11f1p_sd15_depth.pth",
                "scale": 0.5
            }
        ]
    },
    "data": {
        "input_frames": {
            "video_file": "/content/input_video.mp4",
            "image_folder": None,
            "height": 720,
            "width": 480,
            "start_frame_id": 31,
            "end_frame_id": 61
        },
        "controlnet_frames": [
            {
                "video_file": "/content/input_video.mp4",
                "image_folder": None,
                "height": 720,
                "width": 480,
                "start_frame_id": 31,
                "end_frame_id": 61
            },
            {
                "video_file": "/content/input_video.mp4",
                "image_folder": None,
                "height": 720,
                "width": 480,
                "start_frame_id": 31,
                "end_frame_id": 61
            }
        ],
        "output_folder": "data/examples/diffutoon_edit/color_video",
        "fps": 25
    },
    "smoother_configs": [
        {
            "processor_type": "FastBlend",
            "config": {}
        }
    ],
    "pipeline": {
        "seed": 0,
        "pipeline_inputs": {
            "prompt": "best quality, perfect anime illustration, orange clothes, night, a girl is dancing, smile, solo, black silk stockings",
            "negative_prompt": "verybadimagenegative_v1.3",
            "cfg_scale": 7.0,
            "clip_skip": 1,
            "denoising_strength": 0.9,
            "num_inference_steps": 20,
            "animatediff_batch_size": 8,
            "animatediff_stride": 4,
            "unet_batch_size": 8,
            "controlnet_batch_size": 8,
            "cross_frame_attention": True,
            "smoother_progress_ids": [-1],
            # The following parameters will be overwritten. You don't need to modify them.
            "input_frames": [],
            "num_frames": 30,
            "width": 480,
            "height": 720,
            "controlnet_frames": []
        }
    }
}

from diffsynth import SDVideoPipelineRunner

config_stage_1 = config_stage_1_template.copy()
config_stage_1["data"]["input_frames"] = {
    "video_file": "/content/input_video.mp4",
    "image_folder": None,
    "height": 720,
    "width": 480,
    "start_frame_id": 31,
    "end_frame_id": 61
}
config_stage_1["data"]["controlnet_frames"] = [config_stage_1["data"]["input_frames"], config_stage_1["data"]["input_frames"]]
config_stage_1["data"]["output_folder"] = "/content/color_video"
config_stage_1["data"]["fps"] = 25
config_stage_1["pipeline"]["pipeline_inputs"]["prompt"] = "best quality, perfect anime illustration, orange clothes, night, a girl is dancing, smile, solo, black silk stockings"

runner = SDVideoPipelineRunner()
runner.run(config_stage_1)

In [ ]:
import shutil
import os

# 定义源文件路径和目标文件路径
source_file = '/content/color_video/video.mp4'
destination_dir = '/content/drive/MyDrive/test_auto_mv'
destination_file = os.path.join(destination_dir, 'video1.mp4')

# 确保目标目录存在，如果不存在则创建
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# 移动并重命名文件
shutil.move(source_file, destination_file)

print(f'File moved to {destination_file}')

In [ ]:
config_stage_2_template = {
    "models": {
        "model_list": [
            "models/stable_diffusion/aingdiffusion_v12.safetensors",
            "models/AnimateDiff/mm_sd_v15_v2.ckpt",
            "models/ControlNet/control_v11f1e_sd15_tile.pth",
            "models/ControlNet/control_v11p_sd15_lineart.pth"
        ],
        "textual_inversion_folder": "models/textual_inversion",
        "device": "cuda",
        "lora_alphas": [],
        "controlnet_units": [
            {
                "processor_id": "tile",
                "model_path": "models/ControlNet/control_v11f1e_sd15_tile.pth",
                "scale": 0.5
            },
            {
                "processor_id": "lineart",
                "model_path": "models/ControlNet/control_v11p_sd15_lineart.pth",
                "scale": 0.5
            }
        ]
    },
    "data": {
        "input_frames": {
            "video_file": "/content/input_video.mp4",
            "image_folder": None,
            "height": 1024,
            "width": 1024,
            "start_frame_id": 0,
            "end_frame_id": 30
        },
        "controlnet_frames": [
            {
                "video_file": "/content/input_video.mp4",
                "image_folder": None,
                "height": 1024,
                "width": 1024,
                "start_frame_id": 0,
                "end_frame_id": 30
            },
            {
                "video_file": "/content/input_video.mp4",
                "image_folder": None,
                "height": 1024,
                "width": 1024,
                "start_frame_id": 0,
                "end_frame_id": 30
            }
        ],
        "output_folder": "/content/output",
        "fps": 25
    },
    "pipeline": {
        "seed": 0,
        "pipeline_inputs": {
            "prompt": "best quality, perfect anime illustration, light, a girl is dancing, smile, solo",
            "negative_prompt": "verybadimagenegative_v1.3",
            "cfg_scale": 7.0,
            "clip_skip": 2,
            "denoising_strength": 1.0,
            "num_inference_steps": 10,
            "animatediff_batch_size": 16,
            "animatediff_stride": 8,
            "unet_batch_size": 1,
            "controlnet_batch_size": 1,
            "cross_frame_attention": False,
            # The following parameters will be overwritten. You don't need to modify them.
            "input_frames": [],
            "num_frames": 30,
            "width": 1536,
            "height": 1536,
            "controlnet_frames": []
        }
    }
}

from diffsynth import SDVideoPipelineRunner

config_stage_2 = config_stage_2_template.copy()
config_stage_2["data"]["input_frames"] = {
    "video_file": "/content/input_video.mp4",
    "image_folder": None,
    "height": 1024,
    "width": 1024,
    "start_frame_id": 0,
    "end_frame_id": 30
}
config_stage_2["data"]["controlnet_frames"][0] = {
    "video_file": "/content/color_video/video.mp4",
    "image_folder": None,
    "height": config_stage_2["data"]["input_frames"]["height"],
    "width": config_stage_2["data"]["input_frames"]["width"],
    "start_frame_id": None,
    "end_frame_id": None
}
config_stage_2["data"]["controlnet_frames"][1] = config_stage_2["data"]["input_frames"]
config_stage_2["data"]["output_folder"] = "/content/edit_video"
config_stage_2["data"]["fps"] = 25

runner = SDVideoPipelineRunner()
runner.run(config_stage_2)

In [ ]:
import moviepy.editor
moviepy.editor.ipython_display("/content/edit_video/video.mp4")